In [3]:
import numpy as np
import h5py
import sys

sys.path.append("../src")
from Load_data.prepare_data import *

ModuleNotFoundError: No module named 'h5py'

In [ ]:

def build_or_load_dataset(
    cache_path: str,
    image_folder: str,
    xml_path: str,
    label_mode: str,
    img_size: Tuple[int, int] = (64, 64),
    min_coverage: float = 0.3
):
    """
    label_mode:
      - "binary_any"
      - "destroyed_coverage"
    """

    os.makedirs(os.path.dirname(cache_path), exist_ok=True)

    if os.path.exists(cache_path):
        print(f"⚡ Loading cached dataset: {cache_path}")
        with h5py.File(cache_path, "r") as f:
            X = f["X"][:]
            Y = f["Y"][:]
            filenames = f["filenames"][:].astype(str)
        return X, Y, filenames

    print("🛠️ Building dataset from scratch")

    X, ordered_filenames = load_and_resize_images(
        image_folder,
        size=img_size
    )

    if label_mode == "binary_any":
        raw = parse_cvat_xml_all_labels(xml_path)
        labels = label_Y_binary(raw)

    elif label_mode == "destroyed_coverage":
        labels = parse_destroyed_with_size_check(
            xml_path,
            min_coverage=min_coverage
        )
    else:
        raise ValueError(f"Unknown label_mode: {label_mode}")

    Y = build_label_array(ordered_filenames, labels)

    with h5py.File(cache_path, "w") as f:
        f.create_dataset("X", data=X, compression="gzip")
        f.create_dataset("Y", data=Y, compression="gzip")
        f.create_dataset(
            "filenames",
            data=np.array(ordered_filenames, dtype="S")
        )

    print(f"✅ Dataset cached to {cache_path}")
    return X, Y, ordered_filenames